# Receive bandwidth data extraction

The data is stored in the Prometheus and this script will query prometheus for the specific time slots were a successful call were done, calculate the average used bandwidth and save that as a data point in a csv file.

In [ ]:
import sys
from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
import os
import json
import logging
import helperFunctions as hf
import numpy as np
import requests

hf.setup()

outputFolder = "output_folder/"
outputFile = outputFolder + "SuccessfulCallsUsedReceiveBandwidth.csv"
outputFileValues = outputFolder + "SuccessfulCallsUsedReceiveBandwidthValues.csv"


if not os.path.exists(outputFolder):

   # Create a new directory because it does not exist
   os.makedirs(outputFolder)
   logging.info(f"The directory \"{outputFolder}\" is created!")

if os.path.exists(outputFile):
   os.remove(outputFile)

if os.path.exists(outputFileValues):
   os.remove(outputFileValues)

In [ ]:
callsDf = pd.read_csv(outputFolder + "SuccessfulCallsStartAndEnd.csv")

scenarios = callsDf["scenario"].unique()
for scenario in scenarios:

  scenarioDf = callsDf.loc[callsDf["scenario"] == scenario]
  count = len(scenarioDf)
  logging.info(f"scenarioDf {scenario} has {count} records")

  for index, row in scenarioDf.iterrows():

    roomId = row["roomId"]
    client = row["client"]
    start = row["start"]
    end = row["end"]

    if(client == "CnC"):
      continue

    # Get the bandwidth data for the client
    instance = hf.getInstance(client)
    if(instance == None):
      logging.error(f"Could not find instance for client {client}")
      continue

    start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S.%f")
    if(start < datetime(2023, 1, 6)):
      continue
    start = start.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    end = datetime.strptime(end, "%Y-%m-%d %H:%M:%S.%f")
    end = end.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    query = "/api/v1/query_range?query=irate(node_network_receive_bytes_total{instance=\""+ instance + "\", device=\"eth0\"}[1m]) * 8&start=" + start + "&end=" + end + "&step=15s"

    url = "http://localhost:9090" + query

    response = requests.request("GET", url, headers={}, data={})

    

    if(response.status_code == 200):
      try:
        values = json.loads(response.text)["data"]["result"][0]["values"]
        data = pd.DataFrame(columns=["scenario", "roomId", "client", "time", "bps"], data=[[scenario, roomId, client, x[0], x[1]] for x in values])

        hdr = False  if os.path.isfile(outputFileValues) else True
        data.to_csv(outputFileValues, index=False, mode="a", header=hdr)
        
        # find average of bps
        data["bps"] = data["bps"].astype(float)
        average = data["bps"].mean()

        data = pd.DataFrame(columns=["scenario", "roomId", "client", "bps"], data=[[scenario, roomId, client, average]])


        hdr = False  if os.path.isfile(outputFile) else True
        data.to_csv(outputFile, index=False, mode="a", header=hdr)

      except:
        continue

logging.info("Finished!")
      
